## Flavor Graph - data preprocessing

In [1]:
import pandas as pd
from gensim.models.phrases import Phraser

from src.data_preprocessing.text_preprocessing import (
    preprocess_food_item,
    preprocess_food_list,
)
from src.data_preprocessing.flavor_graph_preprocessing import (
    create_wine_nodes,
    create_food_wine_edges
)

In [2]:
NODES_PATH = '../data/flavor_graph/nodes_191120.csv'
EDGES_PATH = '../data/flavor_graph/edges_191120.csv'
FOOD_LIST_PATH = '../data/list_of_foods.csv'
FOOD_TRIGRAMS_PATH = '../data/food_trigrams.pkl'
FOOD_WINE_PAIRING_PATH = '../data/pairing_top3_new.csv'

food_trigram_model = Phraser.load(FOOD_TRIGRAMS_PATH)

In [3]:
nodes_df = pd.read_csv(NODES_PATH)
nodes_df.head()

,node_id,name,id,node_type,is_hub
0,0,1%_fat_buttermilk,NaN,ingredient,no_hub
1,1,1%_fat_cottage_cheese,NaN,ingredient,no_hub
2,3,10%_cream,NaN,ingredient,no_hub
3,4,100%_bran,NaN,ingredient,no_hub
4,5,10_inch_flour_tortilla,NaN,ingredient,no_hub


In [4]:
mask = (nodes_df["node_type"] == "ingredient") & (nodes_df["is_hub"] == "hub")
hub_df = nodes_df.loc[mask]
print(len(hub_df))
hub_df.head()

416


,node_id,name,id,node_type,is_hub
33,38,abalone,NaN,ingredient,hub
41,50,acorn,NaN,ingredient,hub
48,57,adzuki_bean,NaN,ingredient,hub
49,58,agar,NaN,ingredient,hub
52,61,agave,NaN,ingredient,hub


In [5]:
food_df = pd.read_csv(FOOD_LIST_PATH)
food_list = list(food_df["Food"])

food_list_preprocessed = preprocess_food_list(food_list, food_trigram_model)
food_list_preprocessed[:5]

['cashew', 'asparagus', 'squid', 'tartiflett', 'waffl']

In [6]:
hub_df.loc[:, 'name_norm'] = hub_df['name'].apply(lambda x: preprocess_food_item(x, food_trigram_model))
hub_df.head()

/tmp/ipykernel_10556/588982046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hub_df.loc[:, 'name_norm'] = hub_df['name'].apply(lambda x: preprocess_food_item(x, food_trigram_model))


,node_id,name,id,node_type,is_hub,name_norm
33,38,abalone,NaN,ingredient,hub,abalon
41,50,acorn,NaN,ingredient,hub,acorn
48,57,adzuki_bean,NaN,ingredient,hub,adzukibean
49,58,agar,NaN,ingredient,hub,agar
52,61,agave,NaN,ingredient,hub,agav


In [7]:
# take intersection of food list and hub list
intersection_hub_food = list(set(hub_df['name_norm']).intersection(set(food_list_preprocessed)))
print(len(intersection_hub_food))

intersection_hub_food[:5]

193


['pear', 'asparagus', 'squid', 'quinoa', 'molass']

In [8]:
hub_df_filtered = hub_df[hub_df['name_norm'].isin(intersection_hub_food)]

In [9]:
hub_df_filtered.head()

,node_id,name,id,node_type,is_hub,name_norm
41,50,acorn,NaN,ingredient,hub,acorn
68,77,alfalfa,NaN,ingredient,hub,alfalfa
79,88,almond,NaN,ingredient,hub,almond
125,135,anchovy,NaN,ingredient,hub,anchovi
138,148,anise,NaN,ingredient,hub,anis


In [10]:
# wine pairing data
food_wine_pairing_df = pd.read_csv(FOOD_WINE_PAIRING_PATH, index_col=0)
food_wine_pairing_df.head()

,top1,top2,top3
water,"Pinot Noir, , Niederösterreich, Austria","Bordeaux-style Red Blend, Côtes de Bergerac, ...","Chardonnay, Trento, Northeastern Italy, Italy"
Calamari,"Cabernet Sauvignon-Merlot, Bordeaux-style Red ...","Bordeaux-style Red Blend, Mendoza, Mendoza Pr...","Bordeaux-style Red Blend, Long Island, New Y..."
doughnut,"Red Blends, Red Blends, Lake County, Califor...","Red Blends, Red Blends, Valencia, Levante, S...","Red Blends, Red Blends, Salice Salentino, Sou..."
Marjoram,"Pinot Noir, Chorey-lès-Beaune, Burgundy, France","Pinot Noir, Pays dOc, Languedoc-Roussillon, ...","Zinfandel, Columbia Valley, Washington, US"
mint,"Pinot Noir, Crémant dAlsace, Alsace, France","Red Blends, Red Blends, Vino de la Tierra de C...","Chardonnay, Adelaide Hills, South Australia, ..."


In [11]:
# Add food_name column to food_wine_pairing_df - non-normalized food name
food_wine_pairing_df['food_name'] = None
for index, row in food_wine_pairing_df.iterrows():
    try:
        food_wine_pairing_df.loc[index, 'food_name'] = hub_df_filtered[hub_df_filtered['name_norm'] == index]['name'].values[0]
    except IndexError:
        food_wine_pairing_df.loc[index, 'food_name'] = None

# Drop rows with no food_name i.e. outside of the intersection of food_list and hub_list
print(f"Size before droppng nans: {len(food_wine_pairing_df)}")
food_wine_pairing_df = food_wine_pairing_df.dropna(subset=['food_name'])
print(f"Size after droppng nans: {len(food_wine_pairing_df)}")
food_wine_pairing_df.head()

Size before droppng nans: 375
Size after droppng nans: 158


,top1,top2,top3,food_name
water,"Pinot Noir, , Niederösterreich, Austria","Bordeaux-style Red Blend, Côtes de Bergerac, ...","Chardonnay, Trento, Northeastern Italy, Italy",water
mint,"Pinot Noir, Crémant dAlsace, Alsace, France","Red Blends, Red Blends, Vino de la Tierra de C...","Chardonnay, Adelaide Hills, South Australia, ...",mint
ginger,"Pinot Noir, , Thermenregion, Austria","Chardonnay, Adelaide Hills, South Australia, ...","Chardonnay, Franciacorta, Lombardy, Italy",ginger
banana,"Bordeaux-style Red Blend, Côtes de Bergerac, ...","Bordeaux-style Red Blend, Long Island, New Y...","Chardonnay, Trento, Northeastern Italy, Italy",banana
pate,"Pinot Noir, Pernand-Vergelesses, Burgundy, F...","Chardonnay, Vin de France, France Other, France","Cannonau, Grenache, Cannonau di Sardegna, Sic...",pate


In [12]:
MAX_ID = nodes_df['node_id'].max()
print(f"Max node id: {MAX_ID}")

wine_items = set(food_wine_pairing_df['top1'].unique()) | set(food_wine_pairing_df['top2'].unique()) | set(food_wine_pairing_df['top3'].unique())
print(f"Number of wine items: {len(wine_items)}")

Max node id: 8747
Number of wine items: 121


In [13]:
# ADD WINE ITEMS TO NODES
wine_nodes_df = create_wine_nodes(wine_items, MAX_ID)
display(wine_nodes_df.head())

nodes_with_wine_df = pd.concat([nodes_df, wine_nodes_df], ignore_index=True)
nodes_with_wine_df.to_csv('../data/flavor_graph/nodes_with_wine.csv', index=False)

,node_id,name,node_type,is_hub
0,8748,"Cabernet Sauvignon, Barossa, South Australia,...",wine,wine
1,8749,"Bordeaux-style Red Blend, Pauillac, Bordeaux,...",wine,wine
2,8750,"Bordeaux-style Red Blend, Finger Lakes, New ...",wine,wine
3,8751,"Red Blends, Red Blends, North Coast, Califor...",wine,wine
4,8752,"Nebbiolo, Barbaresco, Piedmont, Italy",wine,wine


In [14]:
edges_df = pd.read_csv(EDGES_PATH)
edges_df.head(3)

,id_1,id_2,score,edge_type
0,5063,6083,0.337742,ingr-ingr
1,244,4620,0.093855,ingr-ingr
2,2253,6753,0.107870,ingr-ingr


In [15]:
# ADD EDGES BETWEEN HUB ITEMS AND WINE ITEMS
food_wine_edges_df = create_food_wine_edges(food_wine_pairing_df, nodes_with_wine_df)
print(f"Number of food-wine edges: {len(food_wine_edges_df)}")

edges_with_wine_df = pd.concat([edges_df, food_wine_edges_df], ignore_index=True)
display(edges_with_wine_df.tail(10))
edges_with_wine_df.to_csv('../data/flavor_graph/edges_with_wine.csv', index=False)

Number of food-wine edges: 469


,id_1,id_2,score,edge_type
147638,3148,8849,0.333333,ingr-wine
147639,4966,8814,1.000000,ingr-wine
147640,4966,8810,0.500000,ingr-wine
147641,4966,8764,0.333333,ingr-wine
147642,345,8796,1.000000,ingr-wine
147643,345,8776,0.500000,ingr-wine
147644,345,8863,0.333333,ingr-wine
147645,7043,8787,1.000000,ingr-wine
147646,7043,8755,0.500000,ingr-wine
147647,7043,8843,0.333333,ingr-wine
